In [ ]:
import os
import requests
import os
import re
from bs4 import BeautifulSoup
import json



In [ ]:
def get_categories(json_data):
    all_categories = []
    if 'text_has_categorie' in json_data['@relations']:
        categories = json_data['@relations']['text_has_categorie']
        for c in categories:
            if 'displayName' in c:
                all_categories.append( c['displayName'].lower() )
    return all_categories

def get_languages(json_data):
    all_lang = []
    if 'text_has_language' in json_data['@relations']:
        languages = json_data['@relations']['text_has_language']
        for l in languages:
            if 'displayName' in l:
                all_lang.append( l['displayName'].lower() )
    return all_lang

def get_owners(json_data):
    all_owners = []
    if 'textdrager_has_owner' in json_data['@relations']:
        owners = json_data['@relations']['textdrager_has_owner']
        for o in owners:
            if 'displayName' in o:
                all_owners.append( o['displayName'].lower() )
    return all_owners



def get_text_carrier(json_data):
    carriers = []
    if 'text_part_of' in json_data['@relations']:
        for c in json_data['@relations']['text_part_of']:
            carriers.append(c['id'])
    return carriers


def intersection(list1, list2):
    intersection = [value for value in list1 if value in list2]
    return intersection

## Data model

## Tekst

* Auteur (niet in de dataset?)
* Titel
* Pagina in boek
* Signatuur
* Genormaliseerde titel
* Incipit


In [ ]:
files = os.listdir('BNM_texts')

books = dict()

out = open( 'teksten.tsv' , 'w' , encoding = 'utf-8')

out.write( f'titel\tdatering\tpagina\tincipit\ttitel_norm\ttaal\n')

count = 0 
for file in files:
    if re.search( r'.txt$' , file ):
        #print(file)

        path = os.path.join('BNM_texts' , file)
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)
        
        categories = find_categories(json_data)
        for c in categories:
            if re.search( 'getijden' , c ):
                text = json_data
                out.write( text['_id'] )
                out.write('\t')
                out.write( text['value'] )
                out.write('\t')
                if text['datering'] is not None:
                    out.write( text['datering'] )
                out.write('\t')
                out.write( text['@paginas'] )
                out.write('\t')
                if 'annotatie_incipit' in text and text['annotatie_incipit'] is not None:
                    out.write( text['annotatie_incipit'][0])
                out.write('\t')
                if 'text_has_norm_title' in json_data['@relations']:
                    norm_title = json_data['@relations']['text_has_norm_title']
                    out.write( norm_title[0]['displayName'] )
                out.write('\t')
                languages = get_languages(json_data)
                out.write( ';'.join(languages) )
                out.write('\t')
                carriers = get_text_carrier(json_data)
                out.write( ';'.join(carriers) )
                for book in carriers:
                    books[book]= books.get(book,0)+1
                out.write('\n')
                

out.close()


## Data model

## Boek

* Signatuur
* Titel
* Plaats
* Datering
* Taal
* Eigenaar
* Plaats boek productie (producent)
* Boekproductent: Kopiist, illustrator, ect. 



In [ ]:
dir = 'BNM_carriers'

out = open( 'tekstdragers.tsv' , 'w' , encoding = 'utf-8')

out.write('id,signatuur,datering\n')

files = os.listdir(dir)


for file in files:


    path = os.path.join( dir , file)
    
    try:
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)

        path = os.path.join('BNM_data' , file)

        if json_data['_id'] in books:

            out.write( json_data['_id'] )
            out.write('\t')
            out.write( json_data['bewaarplaats_en_signatuur'] )
            out.write('\t')
            if json_data['datering'] is not None:
                out.write( json_data['datering'] )
            out.write('\n')
    except:
        print(f'Problem in file {file}')

out.close()


## Eigenaar

In [ ]:
dir = 'BNM_carriers'
out = open( 'eigenaars.tsv' , 'w' , encoding = 'utf-8')

files = os.listdir(dir)


for file in files:
    if re.search( r'txt$' , file ):

        path = os.path.join( dir , file)
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)

        path = os.path.join('BNM_data' , file)

        if json_data['_id'] in books:

            owners = get_owners(json_data)
            for o in owners:
                out.write( f"{json_data['_id']}\t{o}\n" )
            
out.close()